In [134]:
import pickle
import os, csv

In [135]:
with open('dic.pkl', 'rb') as f:
    dic = pickle.load(f)

In [103]:
from openpyxl import load_workbook
from datetime import datetime, timedelta
wb = load_workbook('MXUS0IT.xlsx')
sheet = wb.active 
msci = {}
prev = None
for idx, row in enumerate(list(sheet.rows)):
    if idx <= 2:
        continue
    if prev is None:
        prev = (row[0].value, row[1].value)
        continue
    dt = row[0].value
    if timedelta(days = 0) <= dt - datetime(year = 2000, month = 1, day = 1) and timedelta(days = 0) >= dt - datetime(year = 2009, month = 12, day = 31):
        msci[dt] = ((row[1].value - prev[1]) / prev[1])  / (row[0].value - prev[0]).days
    prev = (row[0].value, row[1].value)
print(msci[datetime(2000, 1, 3, 0, 0)])

0.009976168043008379


In [136]:
for company in dic:
    if (dic[company]['date_list'][-1] - dic[company]['date_list'][0]).days < 0:
        dic[company]['date_list'] = dic[company]['date_list'][::-1]
        dic[company]['sentiment_list'] = dic[company]['sentiment_list'][::-1]

In [137]:
import xlrd
from datetime import datetime, timedelta, date
return_rate_dic = {}
delete_list = ['FALC', 'IDT', 'PRCP']
for company in dic:
    if company in delete_list:
        continue
    return_rate_dic[company] = {}
    wb = xlrd.open_workbook('CompanyData/{}output.xls'.format(company))
    sheet = wb.sheets()[0]
    prev = None
    for idx in range(sheet.nrows):
        if idx <= 2:
            continue
        row = sheet.row(idx)
        try:
            _  = float(row[1].value)
        except:
            continue
        if prev is None:
            prev = (row[0].value, row[1].value)
            continue
        dt = xlrd.xldate_as_datetime(row[0].value, 0)
        if timedelta(days = 0) <= dt - dic[company]['date_list'][0] and timedelta(days = 0) >= dt - datetime(year = 2009, month = 12, day = 31):
            return_rate_dic[company][dt] = ((row[1].value - prev[1]) / prev[1])  / (dt - prev[0]).days
        if timedelta(days = 0) < dt - datetime(year = 2009, month = 12, day = 31):
            break
        prev = (dt, float(row[1].value))
return_rate_dic['AAPL']

{datetime.datetime(2000, 2, 1, 0, 0): -0.03373494105966038,
 datetime.datetime(2000, 2, 2, 0, 0): -0.014339072891334462,
 datetime.datetime(2000, 2, 3, 0, 0): 0.04554071415900955,
 datetime.datetime(2000, 2, 4, 0, 0): 0.04537213000835845,
 datetime.datetime(2000, 2, 7, 0, 0): 0.018711431405749473,
 datetime.datetime(2000, 2, 8, 0, 0): 0.007123322240117615,
 datetime.datetime(2000, 2, 9, 0, 0): -0.01958671394959254,
 datetime.datetime(2000, 2, 10, 0, 0): 0.007769146221795824,
 datetime.datetime(2000, 2, 11, 0, 0): -0.0418500132352436,
 datetime.datetime(2000, 2, 14, 0, 0): 0.021647494929318587,
 datetime.datetime(2000, 2, 15, 0, 0): 0.027522864801442587,
 datetime.datetime(2000, 2, 16, 0, 0): -0.04096635294117652,
 datetime.datetime(2000, 2, 17, 0, 0): 0.006571811379739464,
 datetime.datetime(2000, 2, 18, 0, 0): -0.031556209978894315,
 datetime.datetime(2000, 2, 22, 0, 0): 0.005758436369145999,
 datetime.datetime(2000, 2, 23, 0, 0): 0.021416910143153393,
 datetime.datetime(2000, 2, 24, 

In [139]:
#delete_list = ['FALC', 'IDT', 'PRCP', 'BBOX', 'EVOL', 'CTS', 'FDC', 'NIHD', 'BRKS', 'KODK', 'OSPN', 'AEIS', 'LGL', 'QADA', 'PCMI']
for idx, company in enumerate(dic):
    #if company in delete_list:
    #    continue
    #print(idx, company)
    dic[company]['return_rate'] = []
    for i in range( len(dic[company]['date_list']) - 1):
        tmp_msci = []
        tmp_return_rate = []
        for j in range(3, (dic[company]['date_list'][i + 1] - dic[company]['date_list'][i]).days - 2):
            dt = dic[company]['date_list'][i] + timedelta(days = j)
            if dt in msci:
                tmp_msci.append(msci[dt])
            if dt in return_rate_dic[company]:
                tmp_return_rate.append(return_rate_dic[company][dt])
        try:
            dic[company]['return_rate'].append(sum(tmp_return_rate) / len(tmp_return_rate) - sum(tmp_msci) / len(tmp_msci))
        except:
            print(company)
            break
print(dic['AAPL']['return_rate'])

[0.00013116675371252728, -0.00013582028715635375, -0.002943546996529887, 0.006538710837427114, -0.0005701286528585609, 0.001522549659268951, 0.0026557753849965194, -0.004084162566849938, -0.0002046298346361737, -2.2649663317705036e-05, 0.00036522157287846753, -0.0007433909515229642, 0.003848535165584965, 0.00455677521694958, 0.005949056709476675, -0.0005426853920215038, 0.0005785616985198585, 0.0028816203669699016, 0.002117411121068336, 0.0008171520128530656, -0.0002514862788455832, 0.0010187865559994707, 0.0032034970406241564, 0.0008115384103210666, 0.0031335553591840407, -0.0008763718236736431, -0.0013541041395429046, 0.0021213999841640727, 0.00038277010991610806]


In [140]:
with open('dic.pkl', 'wb') as f:
    pickle.dump(dic, f)

In [143]:
for comp in dic:
    print(dic[comp]['date_list'][:4])

[datetime.datetime(2000, 5, 15, 0, 0), datetime.datetime(2000, 8, 10, 0, 0), datetime.datetime(2000, 11, 14, 0, 0), datetime.datetime(2001, 5, 14, 0, 0)]
[datetime.datetime(2000, 5, 15, 0, 0), datetime.datetime(2000, 8, 14, 0, 0), datetime.datetime(2000, 11, 14, 0, 0), datetime.datetime(2001, 5, 7, 0, 0)]
[datetime.datetime(2000, 5, 11, 0, 0), datetime.datetime(2000, 8, 8, 0, 0), datetime.datetime(2000, 11, 14, 0, 0), datetime.datetime(2001, 5, 1, 0, 0)]
[datetime.datetime(2000, 4, 24, 0, 0), datetime.datetime(2000, 7, 28, 0, 0), datetime.datetime(2000, 10, 31, 0, 0), datetime.datetime(2001, 4, 27, 0, 0)]
[datetime.datetime(2000, 5, 12, 0, 0), datetime.datetime(2000, 8, 14, 0, 0), datetime.datetime(2000, 11, 14, 0, 0), datetime.datetime(2001, 5, 14, 0, 0)]
[datetime.datetime(2000, 5, 16, 0, 0), datetime.datetime(2000, 8, 14, 0, 0), datetime.datetime(2000, 11, 14, 0, 0), datetime.datetime(2001, 5, 15, 0, 0)]
[datetime.datetime(2000, 5, 12, 0, 0), datetime.datetime(2000, 8, 10, 0, 0), da

In [164]:
wb = load_workbook('LAR1999-2010.xlsx')
sheet = wb.active 
LAR = {}
for idx, row in enumerate(list(sheet.rows)):
    if idx == 0:
        continue
    if row[0].value not in LAR:
        LAR[row[0].value] = {}
    dt = row[1].value
    LAR[row[0].value][dt] = row[2].value

In [165]:
LAR

{'PEGA': {'3/31/1999': 0.2792154252301454,
  '6/30/1999': 0.24052002634632877,
  '9/30/1999': 0.18178541883380156,
  '12/31/1999': 0.19158179388916002,
  '3/31/2000': 0.16040776115415442,
  '6/30/2000': 0.28968568140451684,
  '9/30/2000': 0.25993229242169713,
  '12/31/2000': 0.16679789669933842,
  '3/31/2001': 0.1714817022715342,
  '6/30/2001': 0.18299133723455907,
  '9/30/2001': 0.17652150972018404,
  '12/31/2001': 0.16081936804400665,
  '3/31/2002': 0.15266371867761633,
  '6/30/2002': 0.15589303262407392,
  '9/30/2002': 0.159091966939511,
  '12/31/2002': 0.16193847374956793,
  '3/31/2003': 0.174211133493129,
  '6/30/2003': 0.19007420091324204,
  '9/30/2003': 0.1855331742349649,
  '12/31/2003': 0.170824130613659,
  '3/31/2004': 0.15486732818420443,
  '6/30/2004': 0.15147672833185669,
  '9/30/2004': 0.1445324948479891,
  '12/31/2004': 0.14957143960258423,
  '3/31/2005': 0.16693507826272502,
  '6/30/2005': 0.16015052668918398,
  '9/30/2005': 0.1834933439280631,
  '12/31/2005': 0.2001965

In [167]:
for comp in dic:
    dic[comp]['LAR'] = []
    if comp not in LAR:
        print(comp)
        continue
    time_list = list(LAR[comp].keys())
    l = len(time_list)
    idx = 0
    for i,dt in enumerate(dic[comp]['date_list']):
        while idx + 1 < l and dt - datetime.strptime(time_list[idx], '%m/%d/%Y') > timedelta(days = 0):
            idx += 1
        dic[comp]['LAR'].append(LAR[comp][time_list[idx]])

INVE
ZOOM
CACI


In [168]:
with open('dic.pkl', 'wb') as f:
    pickle.dump(dic, f)